In [22]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from rouge import Rouge


In [3]:
model_name = "multi-qa-mpnet-base-dot-v1"

In [4]:
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
github_url = "https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv"
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [6]:
df = df.iloc[:300]

In [7]:
answer_llm = df.iloc[0].answer_llm

In [10]:
embedding_model.encode(answer_llm)[0]

-0.42244682

In [11]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [12]:
evaluations = []
for idx, row in df.iterrows():
    llm_embed = embedding_model.encode(row.answer_llm)
    orig_embed = embedding_model.encode(row.answer_orig)
    evaluations.append(llm_embed.dot(orig_embed))

In [15]:
# 75th percentile of scores
np.percentile(evaluations, 75)

31.674309253692627

In [19]:
def normalize_vector(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [20]:
evaluations = []
for idx, row in df.iterrows():
    llm_embed = normalize_vector(embedding_model.encode(row.answer_llm))
    orig_embed = normalize_vector(embedding_model.encode(row.answer_orig))
    evaluations.append(llm_embed.dot(orig_embed))

In [21]:
# 75th percentile of scores
np.percentile(evaluations, 75)

0.836234912276268

In [23]:
rouge_scorer = Rouge()

In [25]:
scores = rouge_scorer.get_scores(df.iloc[10]['answer_llm'], df.iloc[10]['answer_orig'])[0]

In [26]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [28]:
average_f_score = np.mean([scores['rouge-1']['f'], scores['rouge-2']['f'], scores['rouge-l']['f']])

In [29]:
average_f_score

0.35490034990035496

In [32]:
def rouge_score(row):
    scores = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]
    return {"rouge-1": scores['rouge-1']['f'], "rouge-2": scores['rouge-2']['f'], "rouge-l": scores['rouge-l']['f']}
rouge_scores = df.apply(rouge_score, axis=1)

In [38]:
rouge_scores = pd.DataFrame(rouge_scores.values.tolist())

In [39]:
rouge_scores["rouge-2"].mean()

0.20696501983423318